In [69]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [70]:
train = pd.read_csv('../../data/training.csv')
test = pd.read_csv('../../data/testing.csv')
validation = pd.read_csv('../../data/validation.csv')
vic_data = pd.read_csv('../../data/victoria.csv')

In [71]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

CHARGE = -1
DISCHARGE = 1

In [72]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [73]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed, along with the unused first row """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0.0
    df['dispatch'] = 0.0
    df['revenue'] = 0.0
    df['opening'] = 0.0
    df['closing'] = 0.0
    df['decision1'] = 0
    df['decision2'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    first_row = df.loc[0]
    df = df.drop([0], axis=0) 
    
    return df, first_row

In [74]:
def algo3_modified(df):
    """ Finds optimal charge and discharge time across the dataset """
    """ Modified from Algorithm 3 provided in the Excel file (2021_UMelbAppliedData_Worked) """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)
            LOWER_PERCENTILE = quantile_exc(ser,CHARGING_PERCENTILE)
            HIGHER_PERCENTILE = quantile_exc(ser,DISCHARGING_PERCENTILE)
            
            # charge if the current price is lower than the percentile 
            # and is lower than the mean of the next prices
            # discharge otherwise
            # To make sure outliers don't have a huge effect on the decision
            if (df.at[i,PRICE] <= LOWER_PERCENTILE) & (df.at[i,PRICE] < MULTIPLE*thelist.mean()):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= HIGHER_PERCENTILE) & (df.at[i,PRICE] > MULTIPLE*thelist.mean()):
                df.at[i,'decision1'] = DISCHARGE
                
        else:
            # quantile.exc requires a minimum length, so if the remaining length
            # is less than the minimum length, use df.quantile
            
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)
            LOWER_PERCENTILE = ser.quantile(CHARGING_PERCENTILE)
            HIGHER_PERCENTILE = ser.quantile(DISCHARGING_PERCENTILE)

            if (df.at[i,PRICE] <= LOWER_PERCENTILE) & (df.at[i,PRICE] < MULTIPLE*thelist.mean()):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= HIGHER_PERCENTILE) & (df.at[i,PRICE] > MULTIPLE*thelist.mean()):
                df.at[i,'decision1'] = DISCHARGE
            
    return df

In [75]:
def improve(df):
    """ This function tries to modify the decision by increasing the discharging frequency when it's idle """
    
    df['decision2'] = df['decision1']
    IDLE = 0
    
    for i in range(1, len(df)-1):
        
        # if the current decision is idle
        if df.at[i, 'decision1'] == IDLE:
            
            
            # find the next non-idle decision
            j = i+1
            prev = df.at[i-1, 'decision1']
            next_ = df.at[j, 'decision1']
            
            while next_ == IDLE:
                j += 1
                next_ = df.at[j, 'decision1']
                
            # if the previous decision and the next non-idle decision is the same
            if next_ == prev:
                
                prices = list(df.loc[i:j-1, PRICE])
                mean_price = sum(prices)/len(prices)
                next_price = df.at[j, PRICE]
                
                # Charge at negative prices
                if (mean_price <= 0) & (mean_price < next_price):
                    df.at[i:j-1, 'decision2'] = CHARGE
                    
                # Using the revenue formula, the minimum requirement to earn a revenue
                # by discharging at i and charging at j is MULTIPLE_OF
                elif mean_price > (MULTIPLE_OF*next_price):
                    df.loc[i:j-1, 'decision2'] = DISCHARGE
                
    
    return df

In [76]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = df.at[i, "opening"]
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = (rawPower / 2) * eff
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = round(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)), 0)
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue

def get_power(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,'decision2'] == CHARGE):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)   # FROM checkpoint3
        
    elif (df.at[i,'decision2'] == DISCHARGE):
        raw_power = min(POWER, opening_cap*2)      # FROM Algorithm 2 and 3 and checkpoint3
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = get_power(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [77]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    print("Average Revenue per Year:", df["revenue"].sum() / (len(df)/(48*365)))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discharging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2, first_row = create_df(ori_df)
    
    # Find Optimal Charging and Discharging period
    df3 = algo3_modified(df2)
    
    # Some improvement on algorithm3
    df3 = improve(df3)
    
    # Calculate the revenue
    df = calculate(df3)
    
    # Add unused row back to df
    df.loc[0] = first_row
    df = df.sort_index()
    
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [78]:
# Entire Dataset

# Parameters are selected using brute force approach

# This is the ratio for the difference between idle price
# and non-idle price to have a profit (Used in improve(df))
MULTIPLE_OF = 1.58

# This is the ratio for the difference between curr_price and
# mean lookahead price to have a profit (Used in algo3_modified(df))
MULTIPLE = 1.04
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.34
DISCHARGING_PERCENTILE = 0.71
vic = run_all(vic_data)

Total revenue in the dataset: 121122068.0
Total days in the dataset: 1322.0208333333333
Revenue per day: 91618.8799344438
Average Revenue per Year: 33440891.176071987
Time Complexity for running the entire Algorithm 3: 30.384758949279785s


# Check Submission

In [79]:
import sys
sys.path.append("../../instructions") # path of helper functions
from check import * # import everything from check.py

In [80]:
# Copy the dataset 
submission_whole = vic.copy()

# Rename columns for submission
submission_whole = submission_whole.rename(columns = {'Time (UTC+10)' : 'datetime', 'raw_power' : 'power', 'opening' : 'capacity'})

# Create df similar to submission.csv
submit_columns = ['datetime', 'power', 'capacity']
submission_whole = submission_whole[submit_columns]

# Check submission
checked_df = check_submission(submission_whole)

# Not Normal. Flagged as bug
print("Number of bugs:", len(checked_df[checked_df['flag'] != 0]))

Number of bugs: 0


In [81]:
# Training set
MULTIPLE_OF = 1.58
MULTIPLE = 1.04
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.34
DISCHARGING_PERCENTILE = 0.71
train_vic = run_all(train)


# Copy the dataset 
submission_train = train_vic.copy()

# Rename columns for submission
submission_train = submission_train.rename(columns = {'Time (UTC+10)' : 'datetime', 'raw_power' : 'power', 'opening' : 'capacity'})

# Create df similar to submission.csv
submission_train = submission_train[submit_columns]

# Check submission
checked_df = check_submission(submission_train)

# Not Normal. Flagged as bug
print("Number of bugs:", len(checked_df[checked_df['flag'] != 0]))

Total revenue in the dataset: 98855211.0
Total days in the dataset: 1096.0
Revenue per day: 90196.36040145985
Average Revenue per Year: 32921671.546532847
Time Complexity for running the entire Algorithm 3: 24.950411081314087s
Number of bugs: 0


In [82]:
# Testing set
MULTIPLE_OF = 1.58
MULTIPLE = 1.04
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.34
DISCHARGING_PERCENTILE = 0.71
test_vic = run_all(test)


# Copy the dataset 
submission_test = test_vic.copy()

# Rename columns for submission
submission_test = submission_test.rename(columns = {'Time (UTC+10)' : 'datetime', 'raw_power' : 'power', 'opening' : 'capacity'})

# Create df similar to submission.csv
submission_test = submission_test[submit_columns]

# Check submission
checked_df = check_submission(submission_test)

# Not Normal. Flagged as bug
print("Number of bugs:", len(checked_df[checked_df['flag'] != 0]))

Total revenue in the dataset: 5366141.0
Total days in the dataset: 42.0
Revenue per day: 127765.26190476191
Average Revenue per Year: 46634320.5952381
Time Complexity for running the entire Algorithm 3: 1.0037710666656494s
Number of bugs: 0


In [83]:
# Validation set
MULTIPLE_OF = 1.58
MULTIPLE = 1.04
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.34
DISCHARGING_PERCENTILE = 0.71
validation_vic = run_all(validation)


# Copy the dataset 
submission_validation = validation_vic.copy()

# Rename columns for submission
submission_validation = submission_validation.rename(columns = {'Time (UTC+10)' : 'datetime', 'raw_power' : 'power', 'opening' : 'capacity'})

# Create df similar to submission.csv
submission_validation = submission_validation[submit_columns]

# Check submission
checked_df = check_submission(submission_validation)

# Not Normal. Flagged as bug
print("Number of bugs:", len(checked_df[checked_df['flag'] != 0]))

Total revenue in the dataset: 16567124.0
Total days in the dataset: 181.0
Revenue per day: 91531.07182320442
Average Revenue per Year: 33408841.215469614
Time Complexity for running the entire Algorithm 3: 4.2180140018463135s
Number of bugs: 0


In [84]:
submission_whole.to_csv('../../submissions/submit_entire_dataset.csv')
submission_train.to_csv('../../submissions/submit_train.csv')
submission_test.to_csv('../../submissions/submit_test.csv')
submission_validation.to_csv('../../submissions/submit_validation.csv')